<a href="https://colab.research.google.com/github/archanags001/tensorflow/blob/main/Training__Validation_with_ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and Validation Set with ImageDataGenerator

In [2]:
# Run the code below to download the compressed dataset horse-or-human.zip
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2022-04-01 18:53:26--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 142.250.152.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   195MB/s    in 0.7s    

2022-04-01 18:53:26 (195 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
# Download the validation set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2022-04-01 18:53:26--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.198.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip’

validation-horse-or 100%[===================>]  10.95M  --.-KB/s    in 0.07s   

2022-04-01 18:53:27 (164 MB/s) - ‘validation-horse-or-human.zip’ saved [11480187/11480187]



In [4]:
import zipfile
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [5]:
# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [6]:
# Unzip validation set
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [7]:
# Directory with training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

In [8]:
# Directory with validation horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [9]:
# list of filenames in the horses training directories
train_horse_names = os.listdir(train_horse_dir)

# list of filenames in the humans training directories
train_human_names = os.listdir(train_human_dir)


In [10]:
# list of filenames in the horses validation directories
validation_horse_names = os.listdir(validation_horse_dir)

# list of filenames in the human validation directories
validation_human_names = os.listdir(validation_human_dir)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [13]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [14]:
# image generator and Normalize image
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.


In [15]:
# Flow validation images in batches of 128 using validation_datagen generator
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # This is the source directory for validation images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='binary')

Found 256 images belonging to 2 classes.


In [17]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 0.2141 - accuracy: 0.9255 - val_loss: 0.7440 - val_accuracy: 0.8516
Epoch 2/15
8/8 [==============================] - 9s 1s/step - loss: 0.1352 - accuracy: 0.9522 - val_loss: 1.0303 - val_accuracy: 0.8477
Epoch 3/15
8/8 [==============================] - 9s 1s/step - loss: 0.2332 - accuracy: 0.9121 - val_loss: 0.7480 - val_accuracy: 0.8516
Epoch 4/15
8/8 [==============================] - 9s 1s/step - loss: 0.4382 - accuracy: 0.8877 - val_loss: 0.7868 - val_accuracy: 0.8320
Epoch 5/15
8/8 [==============================] - 8s 1s/step - loss: 0.0777 - accuracy: 0.9744 - val_loss: 0.7295 - val_accuracy: 0.8906
Epoch 6/15
8/8 [==============================] - 9s 1s/step - loss: 0.1273 - accuracy: 0.9511 - val_loss: 1.7990 - val_accuracy: 0.7617
Epoch 7/15
8/8 [==============================] - 9s 1s/step - loss: 0.0438 - accuracy: 0.9844 - val_loss: 1.0581 - val_accuracy: 0.8633
Epoch 8/15
8/8 [========================